In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2DTranspose, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split



# Load one of your preprocessed images
image_path = "/content/drive/MyDrive/Bmasks/Bmask_green_highlighted_998002_sat_50.jpg"  # Replace with the path to one of your images
image = cv2.imread(image_path)


if os.path.isfile(image_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    print(f"Height: {height}, Width: {width}")
else:
    print(f"Image file does not exist at: {image_path}")




Height: 256, Width: 256


In [ ]:
import os
import cv2
import numpy as np
import concurrent.futures
import shutil
import tensorflow as tf

# Set the GPU memory growth to avoid GPU memory issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Define input and output folders
input_folder = "/content/drive/MyDrive/preprocessing_multiprocessingmodule"
output_folder = "/content/drive/MyDrive/Bmasks"

if os.path.exists(output_folder):
    # Remove the contents of the output folder
    shutil.rmtree(output_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to create binary masks
def create_binary_mask(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    binary_mask = (green_mask > 0).astype(np.uint8) * 255

    mask_path = os.path.join(output_folder, f"Bmask_{os.path.basename(image_path)}")
    cv2.imwrite(mask_path, binary_mask)
    print(f"Created mask: {os.path.basename(mask_path)}")

# Process all images to create binary masks
image_paths = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith(".jpg")]
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(create_binary_mask, image_paths)

print("Binary mask creation complete.")


Streaming output truncated to the last 5000 lines.
Created mask: Bmask_green_highlighted_202277_sat_22.jpg
Created mask: Bmask_green_highlighted_182422_sat_67.jpg
Created mask: Bmask_green_highlighted_174980_sat_34.jpg
Created mask: Bmask_green_highlighted_134465_sat_44.jpg
Created mask: Bmask_green_highlighted_126796_sat_66.jpg
Created mask: Bmask_green_highlighted_122104_sat_80.jpg
Created mask: Bmask_green_highlighted_162310_sat_47.jpg
Created mask: Bmask_green_highlighted_148381_sat_20.jpg
Created mask: Bmask_green_highlighted_20187_sat_88.jpg
Created mask: Bmask_green_highlighted_20187_sat_85.jpg
Created mask: Bmask_green_highlighted_182422_sat_25.jpg
Created mask: Bmask_green_highlighted_15573_sat_38.jpg
Created mask: Bmask_green_highlighted_182422_sat_70.jpg
Created mask: Bmask_green_highlighted_192918_sat_76.jpg
Created mask: Bmask_green_highlighted_15573_sat_76.jpg
Created mask: Bmask_green_highlighted_13415_sat_26.jpg
Created mask: Bmask_green_highlighted_15573_sat_06.jpg
Cre

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

# Define paths
input_folder = "/content/drive/MyDrive/preprocessing_multiprocessingmodule"
output_folder = "/content/drive/MyDrive/green_vegetation_dataset"
labels_csv = "/content/drive/MyDrive/green_vegetation_labels.csv"

# Create output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List the preprocessed image files
image_files = [f for f in os.listdir(input_folder) if f.startswith("green_highlighted_")]

# Create a list to store image paths and corresponding labels
data = []

for image_file in image_files:
    # Extract the label from the image file name
    label = 1 if "green" in image_file else 0

    # Create the full path to the image
    image_path = os.path.join(input_folder, image_file)

    # Append the image path and label to the data list
    data.append((image_path, label))

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Save the dataset as a CSV file
df = pd.DataFrame(data, columns=["Image_Path", "Label"])
df.to_csv(labels_csv, index=False)

# Copy images to their respective folders (train, validation, test)
for dataset, folder in [(train_data, 'train'), (validation_data, 'validation'), (test_data, 'test')]:
    output_dataset_folder = os.path.join(output_folder, folder)

    # Create the output dataset folder if it doesn't exist
    if not os.path.exists(output_dataset_folder):
        os.makedirs(output_dataset_folder)

    for image_path, label in dataset:
        # Copy the image to the output folder
        image_filename = os.path.basename(image_path)
        output_image_path = os.path.join(output_dataset_folder, image_filename)
        shutil.copy(image_path, output_image_path)


In [ ]:
#augmentation korboo

from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
# Data augmentation settings
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Rescale pixel values to [0, 1]
    rotation_range=20,         # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,     # Randomly shift the width by up to 20% of the image width
    height_shift_range=0.2,    # Randomly shift the height by up to 20% of the image height
    shear_range=0.2,           # Shear intensity (shear angle in radians)
    zoom_range=0.2,            # Randomly zoom into images by up to 20%
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill mode for pixel values outside the image boundaries
)

# Create dataframes for training and validation data
df_train = pd.DataFrame(train_data, columns=["Image_Path", "Label"])
df_validation = pd.DataFrame(validation_data, columns=["Image_Path", "Label"])

# Create data generators
batch_size = 32
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='raw',
    subset='training'
)


Found 3268 validated image filenames.


In [ ]:
validation_generator = train_datagen.flow_from_dataframe(
    df_validation,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='raw',
    subset='validation'
)


Found 0 validated image filenames.


In [ ]:
# Rest of the code

# Create a ResNet-50 model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define the number of training and validation steps per epoch
train_steps_per_epoch = len(df_train) // batch_size
validation_steps_per_epoch = len(df_validation) // batch_size



In [ ]:
# Train the model
epochs = 10  # You can adjust the number of epochs as needed
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch,
    epochs=epochs
)




Epoch 1/10
102/102 [==============================] - 113s 591ms/step - loss: 0.0049 - accuracy: 0.9988
Epoch 2/10
102/102 [==============================] - 64s 619ms/step - loss: 3.8671e-16 - accuracy: 1.0000
Epoch 3/10
102/102 [==============================] - 61s 592ms/step - loss: 3.6571e-13 - accuracy: 1.0000
Epoch 4/10
102/102 [==============================] - 61s 590ms/step - loss: 2.0668e-15 - accuracy: 1.0000
Epoch 5/10
102/102 [==============================] - 63s 609ms/step - loss: 5.5790e-15 - accuracy: 1.0000
Epoch 6/10
102/102 [==============================] - 61s 599ms/step - loss: 5.2588e-14 - accuracy: 1.0000
Epoch 7/10
102/102 [==============================] - 60s 586ms/step - loss: 2.8866e-09 - accuracy: 1.0000
Epoch 8/10
102/102 [==============================] - 60s 588ms/step - loss: 2.4670e-15 - accuracy: 1.0000
Epoch 9/10
102/102 [==============================] - 61s 601ms/step - loss: 6.7357e-14 - accuracy: 1.0000
Epoch 10/10
102/102 [===================

In [ ]:
# Specify the path to save the trained model
model_save_path = "/content/drive/MyDrive/GreenDetection_Resnet50_model.h5"

# Save the trained model
model.save(model_save_path)


In [ ]:
# Assuming you have a test dataset directory and batch size defined
test_data_dir = "/content/drive/MyDrive/green_vegetation_dataset/test"

test_datagen = ImageDataGenerator(rescale=1./255)  # Use the same normalization as for training

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'  # Set to 'binary' for binary classification
)


Found 0 images belonging to 0 classes.


In [ ]:
from tensorflow.keras.models import load_model

model_save_path = "/content/drive/MyDrive/GreenDetection_Resnet50_model.h5"
model = load_model(model_save_path)


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


ValueError: ignored